# Point to point low-thrust transfer

In [248]:
import pykep as pk
import numpy as np
import time
import pygmo as pg
import pygmo_plugins_nonfree as ppnf

from matplotlib import pyplot as plt

In [249]:
class direct_point2point:
    """Represents a direct transcription of the optimal low-thrust transfer between two fixed points.

    This problem works using a Sims-Flanagan leg and manipulating its transfer time T, final mass mf and the controls.

    The decision vector is::

        z = [tof, mf, throttles]

    where throttles is a vector [u0x, u0y,u0z, ...]

    """

    def __init__(
        self,
        rvs=[
            np.array([1, 0.1, -0.1]) * pk.AU,
            np.array([0.2, 1, -0.2]) * pk.EARTH_VELOCITY,
        ],
        rvf=[
            np.array([-1.2, -0.1, 0.1]) * pk.AU,
            np.array([0.2, -1.023, 0.44]) * pk.EARTH_VELOCITY,
        ],
        ms=1000,
        mu=pk.MU_SUN,
        max_thrust=0.12,
        isp=3000,
        tof_bounds=[80, 400],
        mf_bounds=[1000.0, 200.0],
        nseg=10,
        cut=0.6,
    ):
        # We add as data member one single Sims-Flanagan leg using the problem data
        # and some temporary (and unused, thus irrelelvant) values for the to-be-optimzed parameters throttles, tof and mf.
        throttles = np.random.uniform(-1, 1, size=(nseg * 3))
        self.leg = pk.leg.sims_flanagan(
            rvs=rvs,
            ms=ms,
            throttles=throttles,
            rvf=rvf,
            mf=np.mean(mf_bounds),
            tof=np.mean(tof_bounds) * pk.DAY2SEC,
            max_thrust=max_thrust,
            isp=isp,
            mu=mu,
            cut=cut,
        )
        self.tof_bounds = tof_bounds
        self.mf_bounds = mf_bounds

    def get_bounds(self):
        lb = [self.tof_bounds[0], self.mf_bounds[0]] + [-1, -1, -1] * self.leg.nseg
        ub = [self.tof_bounds[1], self.mf_bounds[1]] + [1, 1, 1] * self.leg.nseg
        return (lb, ub)

    def fitness(self, x):
        # 1 - We set the leg using data in the decision vector
        self.leg.tof = x[0] * pk.DAY2SEC
        self.leg.mf = x[1]
        self.leg.throttles = x[2:]
        obj = -x[1]

        # 2 - We compute the constraints violations (mismatch+throttle)
        ceq = self.leg.compute_mismatch_constraints()
        cineq = self.leg.compute_throttle_constraints()
        retval = np.array([obj] + ceq + cineq)  # here we can sum lists

        # 3 - We scale the values in nd units (numerical solvers are sensitive to well-scaled values)
        retval[1:4] /= pk.AU
        retval[4:7] /= pk.EARTH_VELOCITY
        retval[7] /= 1000
        
        return retval

    def get_nec(self):
        return 7

    def get_nic(self):
        return self.leg.nseg

In [250]:
# Problem data
mu = pk.MU_SUN
max_thrust = 0.12
isp = 3000

# Initial state
ms = 1500.0
rs = np.array([1, 0., -0.]) * pk.AU
vs = np.array([0.0, 1, -0.0]) * pk.EARTH_VELOCITY

# Final state
mf = 1300.0
rf = np.array([-0.0, 1.0, 0.]) * pk.AU
vf = np.array([-1, -0, 0]) * pk.EARTH_VELOCITY

# Throttles and tof
nseg = 20
throttles = np.random.uniform(-1, 1, size=(nseg * 3))
tof = 90.0 * pk.DAY2SEC

sf = pk.leg.sims_flanagan(rvs = [rs, vs], ms = ms, throttles = throttles, rvf = [rf, vf], mf = mf, tof = tof, max_thrust = max_thrust, isp = isp, mu=mu, cut = 0.6)

In [252]:
snopt72 = "/usr/local/lib/libsnopt7_c.so"
uda = ppnf.snopt7(library=snopt72, minor_version=2, screen_output=True)
uda.set_integer_option("Major iterations limit", 700)
uda.set_integer_option("Iterations limit", 20000)
uda.set_numeric_option("Major optimality tolerance", 1e-3)
uda.set_numeric_option("Major feasibility tolerance", 1e-11)
algo = pg.algorithm(uda)

In [253]:
prob = pg.problem(udp)
pop = pg.population(prob,1)
pop = algo.evolve(pop)


   SNOPT  C interface  2.0.0   
 S N O P T  7.2-4    (Jun 2006)
 
 SNMEMA EXIT 100 -- finished successfully
 SNMEMA INFO 104 -- memory requirements estimated
 

 Nonlinear constraints      27     Linear constraints       0
 Nonlinear variables        62     Linear variables         0
 Jacobian  variables        62     Objective variables     62
 Total constraints          27     Total variables         62
 

 
 The user has defined       0   out of    1736   first  derivatives
 

        Minor   QPmult   nInf   SumInf   rgNorm  Elastic QPobj
          100  6.9E+01      5  6.2E+00           7.5281412E+04

 Major Minors     Step   nCon Feasible  Optimal  MeritFunction    nS Penalty
     0    153               1  1.4E-02  6.0E-01  9.3947490E+04     2           r i
     1     52  3.0E-02      2  1.4E-02  5.8E-01  9.2988976E+04    30         n rli
     2     14  3.1E-01      4  1.4E-02  4.0E-01  8.5914224E+04    29         n rli
     3     34  1.0E+00      6  1.7E-02  1.7E-02  6.9496045E+04